## An Implementation of Djikstra's Algorithm to a League of Legends Champion

The aim of the project is to develop a full set of solutions to the Aphelios's rotating weapon system problem. In the end, we will create a table where each weapon state (row index) is paired to each other weapon state AND weapon cycle (column index). The only necessary input will be the network edge list, created separately (see ['Aphelios Network Define.xlsx'](https://github.com/paulsylvia20/Djikstras_Algorithm/blob/main/Aphelios%20Network%20Define.xlsx)). I will assume some basic familiarity with [edge lists](https://www.youtube.com/watch?v=83RbL8n3vYU), however, there are plenty of resources online for those who are unfamiliar.

The path finding function maintains three running lists. First, it keeps a tally (dictionary) of all nodes that have been visited so far (visited_nodes). This ensures that when a node and its shortest path are found, the node is never revisited by longer paths. Second, the function maintains a priority list where current paths and their lengths are stored, sorted by length, then expanded to their adjacent nodes (paths). Third is the solutions list where shortest path(s) to a target node along with its length is stored when discovered. This is the final output of the algorithm (solutions).

The original Djikstra's algorithm is capable of finding every shortest path from a start node to every other node in the network. However, to keep the code concise, I will run Djikstra's algorithm once for each start-node -> target-node pairing. Future versions might optimize the algorithm by storing all shortest paths to each other node in the network, given a particular start-node, in a single pass.

In [ ]:
import os
import pandas as pd

#os.chdir("your_path")
edges = pd.read_excel("Aphelios Network Define.xlsx", sheet_name="Complete Network") #dataframe: [[source, target, length],] <- str, str, int

print(edges.head())

  Source Target  Length         Label
0    WRG    RGP       1  Base Network
1    PRG    RGW       1  Base Network
2    BRG    RGW       1  Base Network
3    RWG    WGP       1  Base Network
4    PWG    WGR       1  Base Network


#### Forward Stepping Function Inner
This script is layed out like a Russian nesting doll, from the innermost to outerpost part of Djikstra's algorithm. The innermost function, find_next_steps(), takes a path and spits out a list of all edges that would extend the path one step forward. It does this by opening the edges list and finding any of the edges that have the last node in the provided path as a 'Source' node. It eventally returns the edge list, but not before addressing 2 caveats.

*Caveat 1* <br>
An important element of Djikstra's algorithm is that any nodes stepped-into this way must be marked 'visited.' We accomplish this directly inside of find_next_steps() by examining the output before returning, checking if the "Target" to any of the edges are in visited_nodes, removing them if they have, and marking them in visited_nodes if they haven't. We do this because the basic logic ensures that any visited nodes already have a shortest path. Thus, any future attempts to visit this node must involve longer or equal length paths.

*Caveat 2* <br>
In the standard algorithm, we would apply this logic by (a) checking whether a "Target" node is a member of visited_nodes and, if so, (b) preventing it from ever being visited again. However, the standard algorithm guarantees that we output exactly *one* shortest path for each start_node -> end_node pairing. But we would like to store *all* shortest paths, even if there are two or three of them (i.e., two paths going from start_node to end_node that are equal length). To accomplish this, we instruct the find_next_steps() function to only drop a path if it is in the visited_nodes dictionary AND the current path is longer than the known shortest distance to the start_node. Otherwise, the algorithm proceeds with all equivalent paths traveling together towards end_node. In the end, we will store all equivalent paths as solutions.

In [3]:
# Relevant input data structures
paths = [] # [[length, ['list of nodes']], ...]
visited_nodes = {} # "Node index": shortest dx from start, ...}

# Define a function that takes previous path as input
# Maps out the next steps while updating visited_nodes
def find_next_steps(path, visited_nodes):
    last_node = path[1][-1] # Takes the last node in the current path taken
    
    # Next we generate the next steps. We produce a list from the original edge list wherever the last_node was "Source".
    next_steps = edges[edges["Source"].str.contains(last_node)] #A pandas dataframe similar to the original edge list.

    # Complex Updating of Visited and Next Nodes
    for index, i in next_steps.iterrows(): # Iterate through the next_steps (row entries)
        new_path_length = path[0] + i["Length"] # Determine the new path length by adding edge length to the old path length.
        if i["Target"] in visited_nodes.keys() and new_path_length > visited_nodes[i["Target"]]: #Check whether the new length is equivalent if already visited
            next_steps = next_steps.drop(index) # Drop if longer

        elif i["Target"] not in visited_nodes.keys(): # Add to visited if not visited before
            visited_nodes[i["Target"]] = new_path_length

    return next_steps, visited_nodes


#### Forward Stepping Function Outer
We've implemented the basic process that needs to be conducted in order to properly find next steps along paths and avoid revisiting nodes. Now we have to ensure the steps are developed using the correct paths at the correct times. The aim of paths_one_step_forward() is to simply progress the paths by (a) finding and pulling the shortest running path in paths, (b) stepping it forward along all of its possible edges while marking visited nodes, and (c) replacing it in the queue with all of the new paths. This is the central iteration underlying Djikstra's algorithm:

In [4]:
paths = [] # [[length, ['list of nodes']], ... ]
visited_nodes = {} # {"Node index": shortest dx from start, ...}

def paths_one_step_forward(paths, visited_nodes):
    import heapq

    path = heapq.heappop(paths) # Pull the shortest running path, removing it from the priority list

    next_steps, visited_nodes = find_next_steps(path, visited_nodes) # Identify the next steps and update the visited_nodes list

    # For each next step, add it to the heap
    for ind, j in next_steps.iterrows(): # Find the new full paths and lengths
        new_path = path[1] + [j["Target"]] # <- full path
        new_length = path[0] + j["Length"] # <- int
        heapq.heappush(paths, [new_length, new_path]) # Append new

    return paths, visited_nodes

### Final Implementation

The algorithm is already almost entirely specified. We now just loop paths_one_step forward(). It will progress the search until it satisfies the condition of the loop while handling all necessary management of paths and visited_nodes lists. Since we are less concerned about optimization in this implementation, we will use a while-loop that performs a complete search of the network. At each iteration, we also check paths for solutions. If there is a solution, we store it in solutions, returned at the end. If there are multiple solutions, they will be concatenated into a list.

Note, we have also added the nodes() utility function to enable a simple summarization of the edges list into a complete list of all network nodes.

In [5]:
class Network():
    def __init__(self, edge_list):
        # Here we've specified that the network class takes an edge_list as input which is common convention in network science.
        # Future work can expand the number of data structures taken.
        self.edges = edge_list
    
    # Finally we implement Djikstra's algorithm:
    def path_find(self, start_node, target): #(self, string, string)
        # We start the essential data structures empty, exept for paths which needs to nucleate 
        # with the starting node in place
        visited_nodes = {} #{"Node index": shortest dx from start, ...}
        paths = [[0,[start_node]]] #[[length, ['list of nodes']], ... ]
        solutions = [] #[[length, ['list of nodes']], ... ]; usually 1 or 2 solutions, sometimes 3

        # The while loop conducts the algorithm, iterating paths_one_step_forward()
        # We could conduct the search using one line, but we also have to extract solutions. Hence, the for-loops to follow.
        while len(paths) > 0:
          paths, visited_nodes = paths_one_step_forward(paths, visited_nodes) #steps the algorithm
          
          # Here we extract solutions.
          # The if condition generates a list of the last node in each path, then checks if it is the target
          if target in [paths[index][1][-1] for index, i in enumerate(paths)]: # If there is a solution in paths.
              
              # The list comprehension pulls the paths that end in the target node
              # In other words the for loop loops through the solutions in paths
              for solution in [i for index, i in enumerate(paths) if paths[index][1][-1] == target]:

                # Here we remove the source node from the beginning of solutions.
                # It was important to "nucleate" them, but is unnecessary for the final solution.
                solution = [solution[0], solution[1][1:len(solution[1])]] 
                if solution not in solutions: # Prevents appending any duplicates.
                  solutions.append(solution)
        
        return solutions
    
    # Here we define a function that consolidates nodes into a single list with the associated labels while erasing edges.
    # Labels are helpful to our use case because they can be used to separate nodes by the networks they are members of.
    def nodes(self): 
        import pandas as pd

        # First we erase Source/Target information (since we are erasing edges),
        # and retain a messy "Node" list with duplicates and labels intact.
        temp = pd.concat((edges[["Source", "Label"]].rename(columns={"Source":"Node"}), 
          edges[["Target", "Label"]].rename(columns={"Target":"Node"})))
        
        # Next we leverage groupby() to clean the messy list,
        # creating a list of all of the unique nodes with all labels preserved together as a list.
        nodes = temp.groupby("Node")["Label"].unique()

        return nodes

#### Application
Finally, we apply the algorithm, finding all shortest paths in the network. We avoid starting searches with any of the associational nodes, these are just end points. Instead, we have the 'Base Network' as row indeces and use only these nodes as start points. Then we search from these nodes towards every other node.

In [6]:
aphelios = Network(edges) # Initiate the class instance for the Aphelios project.
nodes = aphelios.nodes() # Extract the whole network as a list of nodes

print(nodes.head())

Node
BGP    [Base Network, Associational Network]
BGR    [Base Network, Associational Network]
BGW    [Base Network, Associational Network]
BPG    [Base Network, Associational Network]
BPR    [Base Network, Associational Network]
Name: Label, dtype: object


In [22]:
# Here we pull a list of nodes that are only part of the base network 
BaseNetwork = [nodes.index[index] for index, i in enumerate(nodes) if "Base Network" in i]
WholeNetwork = nodes.index #and just the whole network

# Here we create a df for the output where the base network is inputs, and whole network is outputs
solutions = pd.DataFrame(data = None,
                         index = BaseNetwork, #inputs
                         columns = WholeNetwork) #outputs

In [23]:
# Since path_finder() returns one solution at time, we iterate through the df one cell at a time.
# To save on computation, we also include a skip for the main diagonal, where the row/column indexes refer to the 
#   same node and no search should be conducted.

for i in range(solutions.shape[0]): # iterate through rows
   for j in range(solutions.shape[1]): # iterate through columns
        if(solutions.index[i] == solutions.columns[j]): # check if the start_node *is* the end_node (along the main diagonal)
            solutions.iloc[i,j] = "" # if so impute a non-value instead
        else:
            solution = aphelios.path_find(solutions.index[i], solutions.columns[j]) # find the shortest paths
            solutions.iloc[i,j] = solution # store the shortest paths

#### Final Output

In [32]:
print(solutions.head())
solutions.to_csv("Aphelios_Paths.csv")

Node                                                BGP  \
BGP                                                       
BGR                         [[4, [GRW, RWB, WBG, BGP]]]   
BGW                         [[4, [GWR, WRB, RBG, BGP]]]   
BPG   [[5, [PGR, GRW, RWB, WBG, BGP]], [5, [PGW, GWR...   
BPR                         [[4, [PRW, RWB, WBG, BGP]]]   

Node                               BGR                                    BGW  \
BGP        [[4, [GPW, PWB, WBG, BGR]]]            [[4, [GPR, PRB, RBG, BGW]]]   
BGR                                               [[4, [GRP, RPB, PBG, BGW]]]   
BGW        [[4, [GWP, WPB, PBG, BGR]]]                                          
BPG   [[5, [PGR, GRW, RWB, WBG, BGR]]]       [[5, [PGW, GWR, WRB, RBG, BGW]]]   
BPR        [[4, [PRW, RWB, WBG, BGR]]]  [[6, [PRG, RGW, GWP, WPB, PBG, BGW]]]   

Node                                                BPG  \
BGP   [[5, [GPR, PRW, RWB, WBP, BPG]], [5, [GPW, PWR...   
BGR                         [[4, [GRW, R

#### One Final Note
It will be helpful to convert these results into a more usable form. The following basically decodes the solutions into directions. The result will help instruct the Aphelios player on the order in which to deplete their weapons to achieve the desired end state.

In [ ]:
methods = solutions.copy()
print(methods.iloc[4,4])
for i in range(methods.shape[0]): # iterate through rows
   for j in range(methods.shape[1]): # iterate through columns
      solutions_set = (methods.iloc[i,j]) # pull the given set of solutions (often only 1 or 2, sometimes 3)
      print(solutions_set)
      if solutions_set == "": # Check if it's an empty solution.
        
        if len(solutions_set[0][1][-1])!=5: # Check if the final node is associational or base.

        # If base...
        # Iterate through the set of solutions, iterate through the nodes in that solution,
        #   gather the last letters of each of the nodes compress them into a string,
        #   then merge them back into a list of methods.
          method = [''.join([node[-1] for node in solution[1]]) for solution in solutions_set]
          methods.iloc[i,j] = method

        # If associational...
        # Do the same, except ignore the associational node at the end (i.e., [0:(len(solution[1])-1)]).
        else:
          method = [''.join([node[-1] for node in solution[1][0:(len(solution[1])-1)]]) for solution in solutions_set]
          methods.iloc[i,j] = method

IndexError: string index out of range

In [ ]:
print(methods.head())
methods.to_csv("Aphelios_Methods.csv")

                          BGP                       BGR  \
BGP                       NaN                    [WBGR]   
BGR                    [WBGP]                       NaN   
BGW                    [RBGP]                    [PBGR]   
BPG            [RWBGP, WRBGP]                   [RWBGR]   
BPR                    [WBGP]                    [WBGR]   
BPW                    [RBGP]                  [GRPBGR]   
BRG                   [PWBGP]            [PWBGR, WPBGR]   
BRP                    [WBGP]                    [WBGR]   
BRW                  [GPRBGP]                    [PBGR]   
BWG                   [PRBGP]                   [RPBGR]   
BWP                    [RBGP]                  [GRWBGR]   
BWR                  [GPWBGP]                    [PBGR]   
GBP            [RWBGP, WRBGP]                   [RWBGR]   
GBR                   [PWBGP]            [PWBGR, WPBGR]   
GBW                   [PRBGP]                   [RPBGR]   
GPB        [RGPWBGP, WGPRBGP]          [RWPBGR, WRPBGR] 